<a href="https://colab.research.google.com/github/javed163/Deep-Learning/blob/main/keras_hyper_parameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns



In [3]:
data = pd.read_csv("/content/drive/MyDrive/Datasets/diabetes.csv")
data.head(2)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0


In [18]:
data.corr()['Outcome']

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [19]:
data.shape

(768, 9)

In [5]:
x = data.iloc[:,:-1].values
y = data.iloc[:,-1].values

In [6]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler() # ranges from -1 to 1
sc.fit_transform(x)

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [90]:
import tensorflow
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential

In [10]:
model = Sequential()
model.add(Dense(32, activation = 'relu', input_dim = 8))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [11]:
model.fit(x_train, y_train, epochs = 10, batch_size = 32, validation_data = (x_test, y_test))

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.6509 - loss: 11.1734 - val_accuracy: 0.5065 - val_loss: 4.5509
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4362 - loss: 5.0145 - val_accuracy: 0.5195 - val_loss: 2.4724
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4895 - loss: 2.8468 - val_accuracy: 0.5844 - val_loss: 1.8885
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5037 - loss: 2.2818 - val_accuracy: 0.6429 - val_loss: 1.4394
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5712 - loss: 1.4732 - val_accuracy: 0.6494 - val_loss: 1.1833
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6150 - loss: 1.2330 - val_accuracy: 0.6169 - val_loss: 1.1238
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5789 - loss: 1.1064 - val_accuracy: 0.6494 - val_loss: 0.9495
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6400 - loss: 0.9498 - val_accuracy: 0.6753 - val_los

In [12]:
# 1. how to select the approprate optimizer
# 2. no of nodes in a layes
# 3. how to select the no of layers
# 4. all in all one model

In [14]:
! pip install -U keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 8.7 MB/s eta 0:00:00


In [16]:
import keras_tuner as kt

In [32]:
def build(hp): # hp is the class object
  model = Sequential()
  model.add(Dense(32, activation = 'relu', input_dim = 8))
  model.add(Dense(1, activation = 'sigmoid'))

  # define the optimizer hyperparameter
  optimizer = hp.Choice('optimizer', values=['adam', 'rmsprop', 'adadelta', 'sgd'])

  model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [33]:
tuner = kt.RandomSearch(build,
                        objective = 'val_accuracy',

                        max_trials = 5)

Reloading Tuner from ./untitled_project/tuner0.json


In [35]:
tuner.search(x_train, y_train, epochs = 5, validation_data = (x_test, y_test))

In [37]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'sgd'}

In [38]:
model = tuner.get_best_models(num_models = 1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [39]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 32)                  │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [41]:
model.fit(x_train, y_train,batch_size = 32, epochs = 20, initial_epoch = 6, validation_data = (x_test, y_test))

Epoch 7/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6999 - loss: 0.6038 - val_accuracy: 0.5779 - val_loss: 0.7421
Epoch 8/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6688 - loss: 0.6182 - val_accuracy: 0.6688 - val_loss: 0.6123
Epoch 9/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7220 - loss: 0.5943 - val_accuracy: 0.6948 - val_loss: 0.5832
Epoch 10/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6964 - loss: 0.5750 - val_accuracy: 0.6753 - val_loss: 0.6060
Epoch 11/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6990 - loss: 0.5825 - val_accuracy: 0.7468 - val_loss: 0.7768
Epoch 12/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6820 - loss: 0.6600 - val_accuracy: 0.7208 - val_loss: 0.5870
Epoch 13/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7169 - loss: 0.5913 - val_accuracy: 0.6169 - val_loss: 0.6703
Epoch 14/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6817 - loss: 0.5884 - val_accuracy: 0.5390 - val_

In [50]:
# techniqu to find the best neurons
def build_model(hp):

  model = Sequential()

  units = hp.Int('units', min_value = 8, max_value = 128, step = 8)

  model.add(Dense(units = units, activation = 'relu', input_dim = 8))
  model.add(Dense(1, activation = 'sigmoid'))

  model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

  return model

In [51]:
tuner = kt.RandomSearch(build_model,
                        objective = 'val_accuracy',
                        max_trials=6,
                        directory = 'mydir',
                        project_name = 'javed')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [52]:
tuner.search(x_train, y_train, epochs = 5, validation_data = (x_test, y_test))

Trial 6 Complete [00h 00m 03s]
val_accuracy: 0.6883116960525513

Best val_accuracy So Far: 0.7337662577629089
Total elapsed time: 00h 00m 17s


In [54]:
tuner.get_best_hyperparameters()[0].values

{'units': 96}

In [57]:
model = tuner.get_best_models(num_models = 1)[0]

In [59]:
model.fit(x_train, y_train, epochs = 10, validation_data = (x_test, y_test))

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5972 - loss: 0.9044 - val_accuracy: 0.6818 - val_loss: 0.6208
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6989 - loss: 0.6045 - val_accuracy: 0.6948 - val_loss: 0.6090
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6634 - loss: 0.6067 - val_accuracy: 0.7013 - val_loss: 0.6628
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6719 - loss: 0.6365 - val_accuracy: 0.6883 - val_loss: 0.6430
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6678 - loss: 0.6089 - val_accuracy: 0.6688 - val_loss: 0.5909
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6331 - loss: 0.6343 - val_accuracy: 0.7078 - val_loss: 0.6229
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6648 - loss: 0.6380 - val_accuracy: 0.6818 - val_loss: 0.6153
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6682 - loss: 0.6615 - val_accuracy: 0.6818 - val_loss:

In [62]:
# how to select no of layers
def build_model(hp):
  model = Sequential()
  model.add(Dense(72, input_dim = 8, activation = 'relu'))
  for i in range(hp.Int('num_layers', min_value = 2, max_value = 10)):
    model.add(Dense(72, activation = 'relu'))
  model.add(Dense(1, activation = 'sigmoid'))

  model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
  return model

In [63]:
tuner = kt.RandomSearch(build_model,
                        objective = 'val_accuracy',
                        max_trials = 3,
                        directory = 'mydir',
                        project_name = 'num_layes')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [67]:
tuner.search(x_train, y_train, epochs = 5, validation_data = (x_test, y_test))

Trial 3 Complete [00h 00m 08s]
val_accuracy: 0.7532467246055603

Best val_accuracy So Far: 0.7532467246055603
Total elapsed time: 00h 00m 16s


In [68]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 10}

In [69]:
model = tuner.get_best_models(num_models = 1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 50 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [70]:
model.fit(x_train, y_train, epochs = 10, validation_data = (x_test, y_test))

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - accuracy: 0.6730 - loss: 0.6195 - val_accuracy: 0.7597 - val_loss: 0.5719
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6672 - loss: 0.6123 - val_accuracy: 0.7273 - val_loss: 0.5531
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6968 - loss: 0.5821 - val_accuracy: 0.7792 - val_loss: 0.5784
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6700 - loss: 0.6118 - val_accuracy: 0.7792 - val_loss: 0.5419
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7058 - loss: 0.5550 - val_accuracy: 0.7143 - val_loss: 0.5950
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6830 - loss: 0.5934 - val_accuracy: 0.6883 - val_loss: 0.5941
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6981 - loss: 0.5695 - val_accuracy: 0.7078 - val_loss: 0.5850
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7347 - loss: 0.5606 - val_accuracy: 0.7403 - val_loss

In [91]:
def build_model(hp):
  model = Sequential()
  counter = 0
  # units = hp.Int('units', )
  model.add(Dense(72, input_dim = 8, activation = 'relu'))
  for i in range(hp.Int('num_layers', min_value = 1, max_value = 10)):
    model.add(Dense(72, activation = 'relu'))
    if(counter==0):
      model.add(Dense(
                      units = hp.Int('units' + str(i), min_value = 8, max_value = 128, step = 8),
                      activation = hp.Choice('activation'+str(i), values = ['relu','sigmoid','tanh']),
                      input_dim = 8
                      ))
      model.add(Dropout(hp.Choice('dropout'+str(i), values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    else:

      model.add(Dense(
                      units = hp.Int('units' + str(i), min_value = 8, max_value = 128, step = 8),
                      activation = hp.Choice('activation'+str(i), values = ['relu','sigmoid','tanh']),
                      )
              )

      counter += 1

  model.add(Dense(1, activation = 'sigmoid'))
  model.compile(optimizer = hp.Choice('optimizer', values = ['adam','nadam','sgd','rmsprop','adadelta']),
                                      loss = 'binary_crossentropy',
                                      metrics = ['accuracy'])
  return model # Added return statement to return the compiled model

In [92]:
tuner = kt.RandomSearch(build_model,
                        objective = 'val_accuracy',
                        max_trials = 3,
                        directory = 'mydir',
                        project_name = 'husen')

Reloading Tuner from mydir/husen/tuner0.json


In [93]:
tuner.search(x_train, y_train, epochs = 5, validation_data = (x_test, y_test))

In [94]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 10,
 'units0': 48,
 'activation0': 'sigmoid',
 'optimizer': 'nadam',
 'units1': 80,
 'activation1': 'relu',
 'units2': 80,
 'activation2': 'tanh',
 'units3': 16,
 'activation3': 'tanh',
 'units4': 40,
 'activation4': 'tanh',
 'units5': 8,
 'activation5': 'relu',
 'units6': 8,
 'activation6': 'relu',
 'units7': 8,
 'activation7': 'relu',
 'units8': 8,
 'activation8': 'relu',
 'units9': 8,
 'activation9': 'relu'}

In [95]:
model = tuner.get_best_models(num_models = 1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'nadam', because it has 2 variables whereas the saved optimizer has 91 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [97]:
model.fit(x_train, y_train, epochs = 10, validation_data = (x_test, y_test))

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6347 - loss: 0.6637 - val_accuracy: 0.6948 - val_loss: 0.5962
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6317 - loss: 0.6461 - val_accuracy: 0.6948 - val_loss: 0.5918
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6215 - loss: 0.6386 - val_accuracy: 0.6948 - val_loss: 0.6170
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6260 - loss: 0.6394 - val_accuracy: 0.6948 - val_loss: 0.6090
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6501 - loss: 0.6248 - val_accuracy: 0.6948 - val_loss: 0.6305
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6503 - loss: 0.6413 - val_accuracy: 0.6948 - val_loss: 0.6035
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6162 - loss: 0.6478 - val_accuracy: 0.6948 - val_loss: 0.5808
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6195 - loss: 0.6329 - val_accuracy: 0.6948 - val_loss: